In [1]:
import gmaps
import os
from sgtaxidata import *
from IPython.display import display,clear_output
import ipywidgets as widgets
from sklearn.ensemble import *
import numpy as np
from datetime import datetime
import time

gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])
dlat,dlon=utils.get_lat_long("Serangoon")

info_box_template = """
<dl>
<dd>{name}</dd>
</dl>
"""

In [2]:
DBNAME="../taxis_large.db"

holidays=['20170626']
def peak(h,m,w):
    return 1 if ((w==1 and (h>=6 and (h <=9 and m<30)) or (h >= 18))) else 0

def weekday(d):
    dt=datetime.strptime(d,"%Y%m%d")
    return 1 if datetime.isoweekday(dt)<6 else 0

In [3]:
fig=None
def draw_map(lat,lon):
    global fig
    if fig:
        fig.close()
    fig=gmaps.figure(center=(1.35,103.8),zoom_level=11)
    location=utils.get_location(lat,lon)
    d={'name':location}
    ml=gmaps.marker_layer([(lat,lon)],info_box_content=[info_box_template.format(**d)])
    fig.add_layer(ml)
    display(fig)
    
    print "Location ",location
    
    
    dates=taxis.get_dates(DBNAME)
    data=[analysis.get_weekday_data(date,DBNAME,lat,lon,holidays=holidays) for date in dates]
    data=[x for x in data if x is not None]
    rf=RandomForestRegressor(n_estimators=100,n_jobs=10,warm_start=True)
    for df in data:
        X=df.drop(['TaxiCount','PeakHour'],axis=1)
        y=np.array(df['TaxiCount'])
        rf.fit(X,y.ravel())
        rf.n_estimators+=20

    h=30
    grid=utils.create_grid(h)
    d=utils.get_taxi_data()
    points=d['features'][0]['geometry']['coordinates']
    for b,a in points:
        i,j=utils.get_grid_cell(a,b,h)
        grid[j][i]+=1

    i,j=utils.get_grid_cell(lat,lon)
    n=datetime.now()
    h=n.hour
    m=int(5 * round(float(n.minute)/5))
    p=1 if datetime.isoweekday(n)<6 else 0
    X=np.array([h,m]).reshape(1,-1)
    
    print "Predicted ",rf.predict(X)[0]
    print "Actual ",grid[j][i]

widgets.interact(draw_map,lat=widgets.FloatSlider(min=constants.min_lat,max=constants.max_lat,step=0.0001,value=dlat,continuous_update=False),
                       lon=widgets.FloatSlider(min=constants.min_lon,max=constants.max_lon,step=0.0001,value=dlon,continuous_update=False))

<function __main__.draw_map>